In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/Extreme Pro 1/'

In [ ]:
yrs=np.arange(1980,2024,1)

if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_jra55_MAX.nc')
    ivt_concat_jra55_polar=ds['ivt_concat_jra55_variable'].values
    iwv_concat_jra55_polar=ds['iwv_concat_jra55_variable'].values
    s_concat_jra55_polar=ds['s_concat_jra55_variable'].values
    
    ivt_concat_jra55_fixed=ds['ivt_concat_jra55_fixed'].values
    iwv_concat_jra55_fixed=ds['iwv_concat_jra55_fixed'].values
    s_concat_jra55_fixed=ds['s_concat_jra55_fixed'].values
    
    ivt_concat_jra55_fixed_poleward=ds['ivt_concat_jra55_fixed_poleward'].values
    iwv_concat_jra55_fixed_poleward=ds['iwv_concat_jra55_fixed_poleward'].values
    s_concat_jra55_fixed_poleward=ds['s_concat_jra55_fixed_poleward'].values
    
for i in range(23,len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
    ivt=ds['ivt']
    
    ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
    iwv=ds['iwv']
    
    ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_polar=ivt_mean
        iwv_concat_jra55_polar=iwv_mean
        s_concat_jra55_polar=s_mean
    else:
        #ivt_concat_jra55_polar=xr.concat([ivt_concat_jra55_polar,ivt_mean],dim='time')
        #iwv_concat_jra55_polar=xr.concat([iwv_concat_jra55_polar,iwv_mean],dim='time')
        #s_concat_jra55_polar=xr.concat([s_concat_jra55_polar,s_mean],dim='time')
        
        ivt_concat_jra55_polar=np.concatenate((ivt_concat_jra55_polar,ivt_mean),axis=0)
        iwv_concat_jra55_polar=np.concatenate((iwv_concat_jra55_polar,iwv_mean),axis=0)
        s_concat_jra55_polar=np.concatenate((s_concat_jra55_polar,s_mean),axis=0)
    
    
    ds=xr.open_dataset(dir3+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_fixed=ivt_mean
        iwv_concat_jra55_fixed=iwv_mean
        s_concat_jra55_fixed=s_mean
    else:
        #ivt_concat_jra55_fixed=xr.concat([ivt_concat_jra55_fixed,ivt_mean],dim='time')
        #iwv_concat_jra55_fixed=xr.concat([iwv_concat_jra55_fixed,iwv_mean],dim='time')
        #s_concat_jra55_fixed=xr.concat([s_concat_jra55_fixed,s_mean],dim='time')
        
        ivt_concat_jra55_fixed=np.concatenate((ivt_concat_jra55_fixed,ivt_mean),axis=0)
        iwv_concat_jra55_fixed=np.concatenate((iwv_concat_jra55_fixed,iwv_mean),axis=0)
        s_concat_jra55_fixed=np.concatenate((s_concat_jra55_fixed,s_mean),axis=0)

    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_fixed_poleward=ivt_mean
        iwv_concat_jra55_fixed_poleward=iwv_mean
        s_concat_jra55_fixed_poleward=s_mean
    else:
        #ivt_concat_jra55_fixed_poleward=xr.concat([ivt_concat_jra55_fixed_poleward,ivt_mean],dim='time')
        #iwv_concat_jra55_fixed_poleward=xr.concat([iwv_concat_jra55_fixed_poleward,iwv_mean],dim='time')
        #s_concat_jra55_fixed_poleward=xr.concat([s_concat_jra55_fixed_poleward,s_mean],dim='time')
        
        ivt_concat_jra55_fixed_poleward=np.concatenate((ivt_concat_jra55_fixed_poleward,ivt_mean),axis=0)
        iwv_concat_jra55_fixed_poleward=np.concatenate((iwv_concat_jra55_fixed_poleward,iwv_mean),axis=0)
        s_concat_jra55_fixed_poleward=np.concatenate((s_concat_jra55_fixed_poleward,s_mean),axis=0)

In [ ]:
dk=xr.Dataset()
dk['ivt_concat_jra55_variable']=(('time','lat','lon'),ivt_concat_jra55_polar)
dk['iwv_concat_jra55_variable']=(('time','lat','lon'),iwv_concat_jra55_polar)
dk['s_concat_jra55_variable']=(('time','lat1','lon1'),s_concat_jra55_polar)
#dk['mfc_concat_merra2_variable']=(('time','lat','lon'),mfc_concat_merra2_polar.values)

dk['ivt_concat_jra55_fixed']=(('time','lat','lon'),ivt_concat_jra55_fixed)
dk['iwv_concat_jra55_fixed']=(('time','lat','lon'),iwv_concat_jra55_fixed)
dk['s_concat_jra55_fixed']=(('time','lat1','lon1'),s_concat_jra55_fixed)
#dk['mfc_concat_merra2_fixed']=(('time','lat','lon'),mfc_concat_merra2_fixed.values)

dk['ivt_concat_jra55_fixed_poleward']=(('time','lat','lon'),ivt_concat_jra55_fixed_poleward)
dk['iwv_concat_jra55_fixed_poleward']=(('time','lat','lon'),iwv_concat_jra55_fixed_poleward)
dk['s_concat_jra55_fixed_poleward']=(('time','lat1','lon1'),s_concat_jra55_fixed_poleward)
#dk['mfc_concat_merra2_fixed_poleward']=(('time','lat','lon'),mfc_concat_merra2_fixed_poleward.values)

dk.coords['time']=np.arange(1980,2024)
dk.coords['lat']=ivt.lat.values
dk.coords['lon']=ivt.lon.values
dk.coords['lat1']=s.lat.values
dk.coords['lon1']=s.lon.values
dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'
dk['lat1'].attrs["units"]='degrees_north'
dk['lon1'].attrs["units"]='degrees_east'
dk['time'].attrs["units"]='years'
print(dk)

try:
    os.remove(dir_data+'ar_vars_geomeans_jra55_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_vars_geomeans_jra55_MAX.nc',mode='w',format='NETCDF4')

In [ ]:
print(ivt_concat_jra55_fixed.time.size)

In [ ]:
if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_jra55_MAX.nc')
    ivt_concat_jra55_variable=ds['ivt_concat_jra55_variable']
    iwv_concat_jra55_variable=ds['iwv_concat_jra55_variable']
    s_concat_jra55_variable=ds['s_concat_jra55_variable']
    
    ivt_concat_jra55_fixed=ds['ivt_concat_jra55_fixed']
    iwv_concat_jra55_fixed=ds['iwv_concat_jra55_fixed']
    s_concat_jra55_fixed=ds['s_concat_jra55_fixed']
    
    ivt_concat_jra55_fixed_poleward=ds['ivt_concat_jra55_fixed_poleward']
    iwv_concat_jra55_fixed_poleward=ds['iwv_concat_jra55_fixed_poleward']
    s_concat_jra55_fixed_poleward=ds['s_concat_jra55_fixed_poleward']

In [ ]:
test=ivt_concat_jra55_fixed[5,:,:]
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=1800,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
print(ivt_concat_jra55_variable[:,50,109].values)
ax.plot(ivt.lon[50],ivt.lat[50],'ko',markersize=12)
plt.show()